<a href="https://colab.research.google.com/github/nangokosu/Python/blob/master/Data_Wrangling_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install pyspark

     |████████████████████████████████| 215.7MB 54kB/s 
     |████████████████████████████████| 204kB 44.6MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=f62f03a39b2c485fa5995341bc9df6a8bfc5e4a8f9d40546a62703486a54fcf5
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


In [0]:
!apt-get install openjdk-8-jdk-headless -qq> /dev/null
import os
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64"

In [0]:
from google.colab import drive
drive.mount("/gdrive")
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [0]:
from pyspark.sql import SparkSession
sparksession=SparkSession.builder.appName('test').getOrCreate()

In [0]:
import pyspark.sql.functions as F
import pyspark.sql.types as T



In [0]:
root_dir = '/gdrive/My Drive/Credit Fraud' # Source: https://www.kaggle.com/janiobachmann/credit-fraud-dealing-with-imbalanced-datasets

In [0]:
df=sparksession.read.csv('/gdrive/My Drive/Credit Fraud',header=True)
#Change to own directory for file
df.show(10, truncate= False)
feature_names=df.columns[1:len(df.columns)-1]
print(feature_names)
print(len(feature_names))

+----+------------------+-------------------+------------------+------------------+-------------------+-------------------+--------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+------------------+-------------------+-------------------+--------------------+-------------------+------+-----+
|Time|V1                |V2                 |V3                |V4                |V5                 |V6                 |V7                  |V8                |V9                |V10                |V11               |V12               |V13               |V14               |V15               |V16                |V17                |V18                |V19                |V20                |V21                 |V22       

In [0]:
df.printSchema()
for feature in df.columns:
   df=df.withColumn(f"{feature}", df[f"{feature}"].cast("float"))
df.printSchema()

root
 |-- Time: string (nullable = true)
 |-- V1: string (nullable = true)
 |-- V2: string (nullable = true)
 |-- V3: string (nullable = true)
 |-- V4: string (nullable = true)
 |-- V5: string (nullable = true)
 |-- V6: string (nullable = true)
 |-- V7: string (nullable = true)
 |-- V8: string (nullable = true)
 |-- V9: string (nullable = true)
 |-- V10: string (nullable = true)
 |-- V11: string (nullable = true)
 |-- V12: string (nullable = true)
 |-- V13: string (nullable = true)
 |-- V14: string (nullable = true)
 |-- V15: string (nullable = true)
 |-- V16: string (nullable = true)
 |-- V17: string (nullable = true)
 |-- V18: string (nullable = true)
 |-- V19: string (nullable = true)
 |-- V20: string (nullable = true)
 |-- V21: string (nullable = true)
 |-- V22: string (nullable = true)
 |-- V23: string (nullable = true)
 |-- V24: string (nullable = true)
 |-- V25: string (nullable = true)
 |-- V26: string (nullable = true)
 |-- V27: string (nullable = true)
 |-- V28: string (nulla

In [0]:
df.select("Class").groupBy("Class").count().show()

+-----+------+
|Class| count|
+-----+------+
|  1.0|   492|
|  0.0|284315|
+-----+------+



In [0]:
df.createOrReplaceTempView("df")

In [0]:
sparksession.sql("select class, count(class)*100/(select count(class) from df) as perc from df group by class").show() #Checking for percentage of fraud to non - fraud transactions in the data set.

+-----+------------------+
|class|              perc|
+-----+------------------+
|  1.0|0.1727485630620034|
|  0.0|   99.827251436938|
+-----+------------------+



In [0]:
from pyspark.ml.feature import Tokenizer, \
StopWordsRemover, \
StringIndexer, \
Word2Vec, \
OneHotEncoderEstimator, \
RegexTokenizer, \
CountVectorizer,\
VectorAssembler


In [0]:
from pyspark.ml.classification import LogisticRegression, MultilayerPerceptronClassifier

In [0]:
Vector_Assembler=VectorAssembler(inputCols=feature_names,outputCol="feature_combined")
dfva=Vector_Assembler.transform(df)
train,test=dfva.randomSplit([0.8,0.2],seed=711)

In [0]:
train.createOrReplaceTempView("train")
sparksession.sql("select class, count(class)*100/(select count(class) from train) as perc from train group by class").show()



+-----+------------------+
|class|              perc|
+-----+------------------+
|  1.0|0.1742427899913537|
|  0.0| 99.82575721000865|
+-----+------------------+



In [0]:
Logistic_Regression=LogisticRegression(maxIter=40, regParam=0.1, labelCol='Class', featuresCol='feature_combined', threshold=0.5)
LRmodel=Logistic_Regression.fit(train)


In [0]:
predictions=LRmodel.transform(test)


In [0]:
result=LRmodel.evaluate(test)

In [0]:
result.accuracy

0.9985956042412751

In [0]:
result.

In [0]:
import numpy as np

In [0]:
layers = [29, 20, 2]

In [0]:
nn_trainer = MultilayerPerceptronClassifier(maxIter=10, layers=layers, seed=1234, featuresCol='feature_combined', labelCol='Class')

In [0]:
nn_fit = nn_trainer.fit(train)

In [0]:
nn_predictions=nn_fit.transform(test)

In [0]:
nn_predictions.select('prediction', 'Class').orderBy(F.rand()).show(10)

+----------+-----+
|prediction|Class|
+----------+-----+
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
+----------+-----+
only showing top 10 rows



In [0]:
nn_predictions.filter(F.col('prediction') == F.col('Class')).count() / nn_predictions.count()

0.9988764833930202

In [0]:
def combine_strigs(t1,t2):
  return t1 + t2

In [0]:
strng_combiner = F.udf(combine_strigs, T.StringType())

In [0]:
df.select(strng_combiner('V1', 'V2')).show()

+----------------------+
|combine_strigs(V1, V2)|
+----------------------+
|  -1.3598071336738-...|
|  1.191857111314860...|
|  -1.35835406159823...|
|  -0.96627171157208...|
|  -1.15823309349523...|
|  -0.42596588441245...|
|  1.229657634507930...|
|  -0.64426944234814...|
|  -0.89428608220282...|
|  -0.33826175242575...|
|  1.44904378114715-...|
|  0.384978215180950...|
|  1.249998742053-1....|
|  1.06937358788190....|
|  -2.7918547659339-...|
|  -0.75241704295660...|
|  1.10321543528383-...|
|  -0.43690507136062...|
|  -5.40125766315825...|
|  1.4929359769862-1...|
+----------------------+
only showing top 20 rows



In [0]:
df.printSchema()

root
 |-- Time: string (nullable = true)
 |-- V1: string (nullable = true)
 |-- V2: string (nullable = true)
 |-- V3: string (nullable = true)
 |-- V4: string (nullable = true)
 |-- V5: string (nullable = true)
 |-- V6: string (nullable = true)
 |-- V7: string (nullable = true)
 |-- V8: string (nullable = true)
 |-- V9: string (nullable = true)
 |-- V10: string (nullable = true)
 |-- V11: string (nullable = true)
 |-- V12: string (nullable = true)
 |-- V13: string (nullable = true)
 |-- V14: string (nullable = true)
 |-- V15: string (nullable = true)
 |-- V16: string (nullable = true)
 |-- V17: string (nullable = true)
 |-- V18: string (nullable = true)
 |-- V19: string (nullable = true)
 |-- V20: string (nullable = true)
 |-- V21: string (nullable = true)
 |-- V22: string (nullable = true)
 |-- V23: string (nullable = true)
 |-- V24: string (nullable = true)
 |-- V25: string (nullable = true)
 |-- V26: string (nullable = true)
 |-- V27: string (nullable = true)
 |-- V28: string (nulla